# Importing Essential Libraries

In [1]:
import os, sys
import pandas as pd
import geopandas as gpd
import shapely
import numpy as np
import bokeh.models as bkhm
import bokeh.colors as bokeh_colors

sys.path.append(os.path.abspath('../..')) # Ideally ST_Visions will be installed as a module, therefore this is more of a development setting for quick testing.

from st_visions.st_visualizer import st_visualizer
import st_visions.express as viz_express
import st_visions.geom_helper as viz_helper
import st_visions.providers as viz_providers
import st_visions.callbacks as viz_callbacks

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*" # DEV ENVIROMENT BANDAID FOR VSCODE VISUALIZATIONS, will be edited

pd.set_option('display.expand_frame_repr', False)

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Defining Global Variables

  * ### Loading Dataset

In [21]:
gdf = pd.read_csv('..\..\data\dummy_categorical_gdf.csv')
gdf = viz_helper.getGeoDataFrame_v2(gdf, crs='epsg:4326')

  * ### Creating Choropleth (Grid) Geometry

  We get the borderbox of an existing GeoDataFrame, then use shapely to create a Polygon covering the box (a square essentially)

In [22]:
# get bbox of an existing GeoDataFrame
bbox = np.array(gdf.total_bounds) 

# Create Square
bbox_poly = shapely.geometry.Polygon([
    (bbox[0], bbox[3]),  # top-left
    (bbox[2], bbox[3]),  # top-right
    (bbox[2], bbox[1]),  # bottom-right
    (bbox[0], bbox[1])   # bottom-left
])

cut_result = viz_helper.quadrat_cut_geometry(bbox_poly, quadrat_width=0.7)

# Covert to GeoDataFrame
parts = list(shapely.get_parts(cut_result))
spatial_coverage_cut = gpd.GeoDataFrame(geometry=parts, crs='EPSG:4326')

print(spatial_coverage_cut)

                                                                                                         geometry
0    POLYGON ((116.36543 39.93567, 116.3941 39.93567, 116.3941 39.91067, 116.36543 39.91067, 116.36543 39.93567))
1    POLYGON ((116.36543 39.91067, 116.3941 39.91067, 116.3941 39.88567, 116.36543 39.88567, 116.36543 39.91067))
2  POLYGON ((116.39411 39.93567, 116.42278 39.93567, 116.42278 39.91067, 116.39411 39.91067, 116.39411 39.93567))
3  POLYGON ((116.39411 39.91067, 116.42278 39.91067, 116.42278 39.88567, 116.39411 39.88567, 116.39411 39.91067))


In [23]:
# get bounding box as (minx, miny, maxx, maxy)
bbox = gdf.total_bounds
west, south, east, north = bbox

polygon_corners = [
    (west, north),
    (east, north),
    (east, south),
    (west, south)
]

bbox_polygon = shapely.geometry.Polygon(polygon_corners)
cut_result = viz_helper.quadrat_cut_geometry(bbox_polygon, 0.7)

spatial_coverage_cut = gpd.GeoDataFrame(
    geometry=list(cut_result.geoms) if hasattr(cut_result, 'geoms') else [cut_result],
    crs='EPSG:4326'
)


  * ### Classifying Area Proximity (i.e., Populate the Choropleth Map)

In [24]:
#Classify points into spatial areas
classified_gdf = viz_helper.classify_area_proximity(gdf.copy(), spatial_coverage_cut, compensate=True, verbose=True)

# Get how many points are in each area
cnt = classified_gdf['area_id'].value_counts()

# Assign counts to spatial_coverage_cut GeoDataFram. Note: spatial_coverage_cut index should match the area_id values
spatial_coverage_cut = spatial_coverage_cut.copy() 
spatial_coverage_cut['count'] = 0 
spatial_coverage_cut.loc[cnt.index, 'count'] = cnt.values

if spatial_coverage_cut.geometry.name != 'geometry':
    spatial_coverage_cut = spatial_coverage_cut.rename_geometry('geometry')


4it [00:00, 499.90it/s]


# 1. Simple Choropleth Map

In [25]:
st_viz = st_visualizer(limit=len(spatial_coverage_cut))
st_viz.set_data(spatial_coverage_cut.dropna())

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)

st_viz.add_numerical_colormap('Viridis256', 'count', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
st_viz.add_polygon(fill_color=st_viz.cmap, line_color=st_viz.cmap, fill_alpha=0.6, muted_alpha=0, legend_label=f'GPS Locations (Choropleth Map)')

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bkhm.WheelZoomTool)

st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

Loading BokehJS ...

# 2. (Advanced) Integrating Filters to Choropleth Map

### Create a VISIONS instance for the Choropleth plot (main canvas)

In [26]:
st_viz = st_visualizer(limit=len(spatial_coverage_cut))
st_viz.set_data(spatial_coverage_cut)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)

st_viz.add_numerical_colormap('Viridis256', 'count', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0), nan_color=bokeh_colors.RGB(1,1,1,0))
st_viz.add_polygon(fill_color=st_viz.cmap, line_color=st_viz.cmap, fill_alpha=0.6, muted_alpha=0, legend_label=f'GPS Locations (Choropleth)')

GlyphRenderer(id='p2597', ...)

### Create a VISIONS instance for keeping (& filtering) the original data

In [27]:
data_points = st_visualizer(limit=len(gdf))
data_points.set_data(gdf)
data_points.set_figure(st_viz.figure)


categorical_name='label'

class Callback(viz_callbacks.BokehFilters):
    def __init__(self, vsn_instance, widget):
        super().__init__(vsn_instance, widget)
        
        
    def callback_prepare_data(self, new_pts, ready_for_output):
        self.vsn_instance.canvas_data = new_pts

        if ready_for_output:
            cnt = viz_helper.classify_area_proximity(self.vsn_instance.canvas_data, st_viz.data, compensate=True, verbose=True).area_id.value_counts()            
            st_viz.canvas_data = st_viz.data.loc[cnt.index].copy()
            st_viz.canvas_data.loc[:, 'count'] = cnt.values               
           
            st_viz.canvas_data = st_viz.prepare_data(st_viz.canvas_data)

            low, high = st_viz.canvas_data[st_viz.cmap['field']].agg(['min', 'max'])
            st_viz.cmap['transform'].low = 0 if low == high else low
            st_viz.cmap['transform'].high = high
                    
            st_viz.source.data = st_viz.canvas_data.drop(st_viz.canvas_data.geometry.name, axis=1).to_dict(orient="list")

            # print ('Releasing Lock...')
            st_viz.canvas_data = None
            self.vsn_instance.canvas_data = None
            self.vsn_instance.aquire_canvas_data = None
        
        
    def callback(self, attr, old, new):
        self.callback_filter_data()

        cat_value = self.widget.value
        new_pts = self.get_data()

        # print (cat_value, categorical_name)
        if cat_value:
            new_pts = new_pts.loc[new_pts[categorical_name] == cat_value].copy()

        self.callback_prepare_data(new_pts, self.widget.id==self.vsn_instance.aquire_canvas_data)

        
data_points.add_categorical_filter(title='Vehicle', categorical_name=categorical_name, height_policy='min', callback_class=Callback)

Select(id='p2604', ...)

### Camera, Lights, Action

In [28]:
data_points.figure.legend.location = "top_left"
data_points.figure.legend.click_policy = "mute"
data_points.figure.toolbar.active_scroll = data_points.figure.select_one(bkhm.WheelZoomTool)

In [ ]:
data_points.show_figures(notebook=True, sizing_mode='stretch_both')

Loading BokehJS ...

4it [00:00, 1000.13it/s]
4it [00:00, 999.77it/s]
4it [00:00, 1000.19it/s]
4it [00:00, 799.71it/s]
4it [00:00, 799.79it/s]
4it [00:00, 1000.13it/s]


# Plot Multiple Datasets at the Same Canvas

In [2]:
df = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv')
df.loc[:, 'datetime'] = pd.to_datetime(df.t, unit='ms')

df_algn_inter = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_jun2017.csv')
df_algn_inter.loc[:, 'datetime'] = pd.to_datetime(df_algn_inter.t, unit='ms')

In [3]:
st_viz = st_visualizer(limit=500)
st_viz.set_data(df.head(500), sp_columns=['lon', 'lat'])

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)


st_viz.add_marker(marker='circle', size=10, color='royalblue', alpha=0.5, fill_alpha=0.5, muted_alpha=0, legend_label=f'GPS Locations')
# st_viz.add_hover_tooltips(tooltips = [('Vessel_ID','@vessel_id'), ('Datetime','@datetime{%Y-%m-%d %H:%M:%S.%3N}'), ('Coordinates','(@lon, @lat)')], formatters={'@datetime': 'datetime'}, renderers=st_viz.renderers)

GlyphRenderer(id='p1054', ...)

In [4]:
st_viz2 = st_visualizer(limit=500)
st_viz2.set_data(df_algn_inter.head(500), sp_columns=['lon', 'lat'])

st_viz2.set_figure(st_viz.figure)
st_viz2.create_source()


st_viz2.add_marker(marker='square', size=10, color='orangered', alpha=0.5, fill_alpha=0.5, muted_alpha=0, legend_label=f'GPS Locations (interpolated - Kafka)')
st_viz2.add_hover_tooltips(tooltips = [('Vessel ID','@vessel_id'), ('Datetime','@datetime{%Y-%m-%d %H:%M:%S.%3N}'), ('Coordinates','(@lon, @lat)')], formatters={'@datetime': 'datetime'}, renderers=st_viz2.renderers)

In [5]:
df_traj = viz_helper.create_linestring_from_points(viz_helper.getGeoDataFrame_v2(df.head(100).copy()), ['vessel_id'])

st_viz3 = st_visualizer(limit=500)
st_viz3.set_data(df_traj, sp_columns=['lon', 'lat'])

st_viz3.set_figure(st_viz2.figure)
st_viz3.create_source()

st_viz3.add_line(line_type='multi_line', line_width=5, line_color='limegreen', alpha=0.5, muted_alpha=0, legend_label=f'Vessel Trajectories')

  0%|          | 0/48 [00:00<?, ?it/s]

c:\Users\Nokashi\Desktop\Thesis\ST-Visions\ST_VisionsEnv\lib\site-packages\tqdm\std.py:917: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return getattr(df, df_function)(wrapper, **kwargs)
100%|██████████| 48/48 [00:00<00:00, 4800.12it/s]


GlyphRenderer(id='p1084', ...)

In [6]:
st_viz3.figure.legend.location = "top_left"
st_viz3.figure.legend.click_policy = "mute"
st_viz3.figure.toolbar.active_scroll = st_viz3.figure.select_one(bkhm.WheelZoomTool)
st_viz3.show_figures(notebook=True, sizing_mode='stretch_both')

Loading BokehJS ...

# Plot Multiple Datasets at Grid

In [ ]:
df = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv')
df.loc[:, 'datetime'] = pd.to_datetime(df.t, unit='ms')

df_algn_inter = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_jun2017.csv')
df_algn_inter.loc[:, 'datetime'] = pd.to_datetime(df_algn_inter.t, unit='ms')

In [ ]:
st_viz = st_visualizer(limit=1000)
st_viz.set_data(df.head(1000))

tooltips = [('Vessel_id','@vessel_id'), ('Datetime','@t'), ('Coordinates','(@lon, @lat)')]
viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

In [ ]:
st_viz2 = st_visualizer(limit=1000)
st_viz2.set_data(df_algn_inter.head(1000))

tooltips = [('Vessel ID','@vessel_id'), ('Datetime','@t'), ('Coordinates','(@lon, @lat)')]
viz_express.plot_points_on_map(st_viz2, tools=['lasso_select'], tooltips=tooltips, marker='triangle', color='red')

In [74]:
st_viz2.figure.legend.location = "top_left"
st_viz2.figure.legend.click_policy = "mute"
st_viz2.figure.toolbar.active_scroll = st_viz2.figure.select_one(bkhm.WheelZoomTool)

In [ ]:
st_viz2.show_figures([[st_viz.figure, st_viz2.figure]], notebook=True, merge_tools=True, toolbar_location='right', sizing_mode='stretch_both')

# Visualize Data with Map alongside DataFrame

In [20]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
viz_providers.add_tile_to_canvas(st_viz)

circ = st_viz.add_marker(marker='circle', size=10, color='royalblue', alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]
st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()
st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bkhm.WheelZoomTool)


columns = [
        bkhm.TableColumn(field="vessel_id", title="Vessel ID"),
        bkhm.TableColumn(field="speed", title="Speed"),
        bkhm.TableColumn(field="heading", title="Heading"),
        bkhm.TableColumn(field="lon", title="Longitude"),
        bkhm.TableColumn(field="lat", title="Latitude"),
        bkhm.TableColumn(field="t", title="Timestamp"),
        bkhm.TableColumn(field="course", title="COG")
        ]
data_table = bkhm.DataTable(source=st_viz.source, columns=columns, width=500, height=280)


st_viz.show_figures([[st_viz.figure, data_table]], width=1900, height=1900, sizing_mode='stretch_both', notebook=True)

Loading BokehJS ...